<a href="https://colab.research.google.com/github/Samgoles/Covid-19-Paper-Rec-Engine./blob/main/Final_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DF=pd.read_csv('/content/drive/MyDrive/BfinalLHL/DF_final.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
import pandas as pd
import zipfile

zf = zipfile.ZipFile('/content/drive/MyDrive/BfinalLHL/Bclean2.zip') 
Bclean2 = pd.read_csv(zf.open('Bclean2.csv'))

In [ ]:
#data_lda = piplinelda20.transform(DF2['clean_abs'])
Df=pd.read_csv('/content/drive/MyDrive/BfinalLHL/Lda20_XX.csv')
DF2=Df[Df.pub_year>=2020].iloc[0:10000,:]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from matplotlib import pyplot
from gensim.models import KeyedVectors

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Average Word2Vec

Splitting the descriptions into words and stored in a list called ‘corpus’ for training our word2vec model

In [ ]:
#splitting the description into words

corpus = []
for words in Df['clean_abs']:
    corpus.append(words.split())

In [ ]:
# Downloading the Google pretrained Word2Vec Model
#!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'
google_word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

# Training our corpus with Google Pretrained Model

google_model = Word2Vec(size = 300, window=5, min_count = 2, workers = -1)
google_model.build_vocab(corpus)

#model.intersect_word2vec_format('./word2vec/GoogleNews-vectors-negative300.bin', lockf=1.0, binary=True)

google_model.intersect_word2vec_format(EMBEDDING_FILE, lockf=1.0, binary=True)

google_model.train(corpus, total_examples=google_model.corpus_count, epochs = 5)

(0, 0)

In [ ]:
import pickle
with open('/content/drive/MyDrive/BfinalLHL/googlemodel.pickle', 'wb') as f:
    pickle.dump(google_model, f)


In [ ]:
del google_model

In [ ]:
import pickle
with open('/content/drive/MyDrive/BfinalLHL/googlemodel.pickle','rb') as f:
    g_model = pickle.load(f)

In [ ]:
g_model

Created a function called vectors for generating average Word2Vec embeddings and stored the same as a list called ‘word_embeddings’. The code follows the steps which I have written in the above-average word2vec explanation.

In [ ]:
# Generate the average word2vec for the each paper abstract

def vectors(x):
    
    # Creating a list for storing the vectors (description into vectors)
    global word_embeddings
    word_embeddings = []

    # Reading the each book description 
    for line in x:
        avgword2vec = None
        count = 0
        for word in line.split():
            if word in g_model.wv.vocab:
                count += 1
                if avgword2vec is None:
                    avgword2vec = g_model[word]
                else:
                    avgword2vec = avgword2vec + g_model[word]
                
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
        
            word_embeddings.append(avgword2vec)


# Recommending the Top 5 similar papers based on title (using word2vec embeding)


In [ ]:
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

In [ ]:
# Recommending the Top 5 similar papers
import html
from IPython.display import HTML
def recommendations0(title):
    
    # Calling the function vectors

    #vectors(DF['clean_abs'])
    
    # finding cosine similarity for the vectors
    indices = pd.Series(DF2.index, index = DF2['title']).drop_duplicates()
         
    idx = indices[title]
    cosine_similarities = cosine_similarity(word_embeddings, word_embeddings)
    print(cosine_similarities.shape)
    # taking the title and ppr link and store in new data frame called pprs
    pprs = DF2[['title', 'url','publish_time']]
    #Reverse mapping of the index

    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]
    book_indices = [i[0] for i in sim_scores]
    recommend = pprs.iloc[book_indices]
    
    recommend.style.format({'url': make_clickable})
    #HTML(recommend.to_html(render_links=True, escape=False))
    return(recommend.sort_values(by='publish_time'))

In [ ]:
vectors(DF2['clean_abs'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
word_embeddings[0].size

300

In [ ]:


recommendations0(DF2['title'].iloc[1000])


(30000, 30000)


,title,url,publish_time
990,Immunological disturbances associated with mal...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,2012-09-27
18671,The Black Box of Nordic Education Held Against...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,2020-10-08
18017,"Agitation, confusion, and aggression in critic...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,2020-12-11
22929,Mental health service use and its associated f...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,2021-04-05
10600,The psychological impact and experience of bre...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,2021-05-08


In [ ]:
recommendations(DF2['title'].iloc[1000])

,title,url,publish_time
144,Apoptotic signals induce specific degradation ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,2008-04-01
26700,SHIP deficiency causes Crohn's disease-like il...,https://doi.org/10.1136/gut.2009.202283; https...,2010-10-12
1562,A co-culture genome-wide RNAi screen with mamm...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,2015-01-09
22722,Taro Lectin Can Act as a Cytokine-Mimetic Comp...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,2021-03-07
9842,Mutant glucocorticoid receptor binding element...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,2021-03-26


In [ ]:
DF2['title'].iloc[1000]

'Infiltration of Proinflammatory M1 Macrophages into the Outer Retina Precedes Damage in a Mouse Model of Age-Related Macular Degeneration'

In [ ]:
DF2.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'cord_uid', 'sha', 'source_x', 'title',
       'doi', 'pmcid', 'pubmed_id', 'license', 'abstract', 'publish_time',
       'authors', 'journal', 'mag_id', 'who_covidence_id', 'arxiv_id',
       'pdf_json_files', 'pmc_json_files', 'url', 's2_id', 'language',
       'pub_year', 'month', 'y-m', 'clean_abs'],
      dtype='object')

In [ ]:
import pickle
with open('/content/drive/MyDrive/BfinalLHL/pipeline_LDA20.pickle','rb') as f:   #loading clean abstract topic modeling
    piplinelda20 = pickle.load(f)

In [ ]:
# Recommending the Top 5 similar papers

def recommendations(title):
    
    # Calling the function vectors

    #vectors(DF['clean_abs'])
    
    # finding cosine similarity for the vectors
    indices = pd.Series(DF2.index, index = DF2['title']).drop_duplicates()
         
    idx = indices[title]
    cosine_similarities = cosine_similarity(DF2.iloc[:,28:49], DF2.iloc[:,28:49])

    # taking the title and ppr link and store in new data frame called pprs
    pprs = DF2[['title', 'url','publish_time']]
    #Reverse mapping of the index

    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]
    book_indices = [i[0] for i in sim_scores]
    recommend = pprs.iloc[book_indices]
    
    recommend.style.format({'url': make_clickable})
    #HTML(recommend.to_html(render_links=True, escape=False))
    return(recommend.sort_values(by='publish_time'))


In [ ]:
len(word_embeddings)

30000

In [ ]:
DF2=DF2.reset_index()
DF2

,index,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,cord_uid,sha,source_x,title,doi,pmcid,...,10,11,12,13,14,15,16,17,18,19
0,3888,3888,3888,4249,f9tg6xsg,44449ad1cca160ce491d7624f8ae1028f3570c45,PMC,Dexmedetomidine improved renal function in pat...,10.1186/s40560-019-0415-z,PMC6939335,...,0.000158,0.043423,0.000158,0.024204,0.835698,0.000158,0.000158,0.000158,0.000158,0.000158
1,3889,3889,3889,4250,f73c639r,def41c08c3cb1b3752bcff34d3aed7f8486e1c86,PMC,Aortic volume determines global end-diastolic ...,10.1186/s40635-019-0284-8,PMC6940405,...,0.000254,0.005704,0.020360,0.037917,0.134629,0.642733,0.046374,0.000254,0.000254,0.000254
2,3890,3890,3890,4251,1qgpa45q,f5ae3f66face323615df39d838e056ab5fcc98df,PMC,Whole genome sequencing and phylogenetic analy...,10.1186/s12864-019-6400-z,PMC6941262,...,0.007521,0.033023,0.018648,0.215623,0.040344,0.000269,0.000269,0.000269,0.155301,0.000269
3,3891,3891,3891,4252,g34f5w6b,5be75ae4e7f8c892abd8dc396b9dbd035772c84a,PMC,European intensive care physicians’ experience...,10.1186/s13756-019-0662-8,PMC6941296,...,0.000148,0.000148,0.020007,0.000148,0.098044,0.000148,0.622550,0.098668,0.000148,0.103591
4,3892,3892,3892,4253,d1pd09zj,1cee4a0d0e823379ec34a462a04561bf4cd736a2,PMC,Synthetic carbohydrate-based vaccines: challen...,10.1186/s12929-019-0591-0,PMC6941340,...,0.013325,0.000538,0.000538,0.012178,0.000538,0.000538,0.585563,0.000538,0.000538,0.080902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,18177,18177,18177,40065,i2b6prxu,9211f31eb7553fe5f6b333ab61236ba2f503d140,PMC,Suicidality in Patients with Brain Tumors: A B...,10.3390/medicina56120725,PMC7767493,...,0.000370,0.142791,0.000370,0.191477,0.120325,0.000370,0.000370,0.024253,0.015293,0.000370
9996,18178,18178,18178,40066,t6ww61va,b597598d727a63d0843f7247c1b36a067cf260e6,PMC,The Antimalarial Compound Atovaquone Inhibits ...,10.3390/v12121475,PMC7767512,...,0.027698,0.029100,0.023712,0.014647,0.000276,0.000276,0.000276,0.000276,0.163339,0.009061
9997,18179,18179,18179,40067,jz1g2ima,7ef05fc08b85f25ce621d117a08db2c6c46b7222,PMC,Computational Analysis of African Swine Fever ...,10.3390/pathogens9121078,PMC7767518,...,0.007049,0.000254,0.030731,0.000254,0.000254,0.000254,0.017359,0.006341,0.203933,0.000254
9998,18180,18180,18180,40068,kzo7ez30,a48ca7314512bc4c6888cafb760488653102a15d; f3df...,PMC,Safety and Efficacy of Intermittent High-Dose ...,10.3390/jof6040385,PMC7767522,...,0.013608,0.051593,0.012676,0.081414,0.310122,0.000239,0.320362,0.000239,0.028613,0.000239


## Final Recommender engine which uses a combination of similarity scores of word2vec Abstract, LDA parameters of Abstract and tfidf of titles. 

In [ ]:
# Recommending the Top 10 similar papers
#DF2=Df.iloc[0:10000,:]
def Recommendations(title):
    
    # Calling the function vectors

    #vectors(DF['clean_abs'])
    
    # finding cosine similarity for the vectors
    indices = pd.Series(DF2.index, index = DF2['title']).drop_duplicates()
         
    idx = indices[title]
    
    # Calculating the similarity measures based on Cosine Similarity
    cosine_similarities1 = cosine_similarity(DF2.iloc[:,28:49], DF2.iloc[:,28:49])#LDA Abstract

    cosine_similarities0 = cosine_similarity(word_embeddings, word_embeddings)   #word2vec Abstract
    #Converting the paper title into vectors and used bigram
    tf = TfidfVectorizer(analyzer='word', ngram_range=(3, 3), min_df = 1, stop_words='english')
    tfidf_matrix = tf.fit_transform(DF2['title'])
    cosine_similarities2 = cosine_similarity(tfidf_matrix, tfidf_matrix)
    cosine_similarities=cosine_similarities0+cosine_similarities1+cosine_similarities2*2
    # taking the title and ppr link and store in new data frame called pprs
    pprs = DF2[['title', 'url','publish_time']]
    #Reverse mapping of the index
    #print((cosine_similarities0))
    #print(cosine_similarities1)
    #print(cosine_similarities2)
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    t=DF2.loc[idx,'abstract']
    print(f'Similar Articles to :\n {title}')#' \n# Abstract: \n {t}')
    sim_scores = sim_scores[1:11]
    pp_indices = [i[0] for i in sim_scores]
    recommend = pprs.iloc[pp_indices]
    
    recommend.style.format({'url': make_clickable})
    #HTML(recommend.to_html(render_links=True, escape=False))
    return(recommend.sort_values(by='publish_time',ascending=False).reset_index(drop=True))

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
#getting the information with doi from the altmetric site.

In [ ]:
def get_request(doi): 
    return f"https://api.altmetric.com/v1/doi/{doi}"
    
calls = DF2['doi'].apply(lambda doi: f"https://api.altmetric.com/v1/doi/{doi}").to_frame()
calls['cord_uid'] = DF2['cord_uid']  
calls.to_csv('/content/drive/MyDrive/BfinalLHL/altmetric_calls.csv', index=False)
del calls


In [ ]:
calls=pd.read_csv('/content/drive/MyDrive/BfinalLHL/altmetric_calls.csv')
calls

,doi,cord_uid
0,https://api.altmetric.com/v1/doi/10.1186/s4056...,f9tg6xsg
1,https://api.altmetric.com/v1/doi/10.1186/s4063...,f73c639r
2,https://api.altmetric.com/v1/doi/10.1186/s1286...,1qgpa45q
3,https://api.altmetric.com/v1/doi/10.1186/s1375...,g34f5w6b
4,https://api.altmetric.com/v1/doi/10.1186/s1292...,d1pd09zj
...,...,...
9995,https://api.altmetric.com/v1/doi/10.3390/medic...,i2b6prxu
9996,https://api.altmetric.com/v1/doi/10.3390/v1212...,t6ww61va
9997,https://api.altmetric.com/v1/doi/10.3390/patho...,jz1g2ima
9998,https://api.altmetric.com/v1/doi/10.3390/jof60...,kzo7ez30


In [ ]:
import requests as re
f = lambda x: re.get(x).text
calls['impact'] = calls.doi.apply(f)   

In [ ]:
calls

,doi,cord_uid,impact
0,https://api.altmetric.com/v1/doi/10.1186/s40560-019-0415-z,f9tg6xsg,"{""title"":""Dexmedetomidine improved renal function in patients with severe sepsis: an exploratory analysis of a randomized controlled trial"",""doi"":""10.1186/s40560-019-0415-z"",""altmetric_jid"":""52d3ea3486a3487b7e8b462f"",""issns"":[""2052-0492""],""journal"":""Journal of Intensive Care"",""cohorts"":{""pub"":3,""doc"":1,""com"":1},""context"":{""all"":{""count"":20485557,""mean"":9.810633031363073,""rank"":11751859,""pct"":42,""higher_than"":8617919},""journal"":{""count"":470,""mean"":10.49308510638298,""rank"":272,""pct"":40,""higher_than"":192},""similar_age_3m"":{""count"":417074,""mean"":13.281103003302887,""rank"":197668,""pct"":51,""higher_than"":215751},""similar_age_journal_3m"":{""count"":51,""mean"":8.168627450980392,""rank"":28,""pct"":47,""higher_than"":24}},""authors"":[""Tsuyoshi Nakashima"",""Kyohei Miyamoto"",""Nozomu Shima"",""Seiya Kato"",""Yu Kawazoe"",""Yoshinori Ohta"",""Takeshi Morimoto"",""Hitoshi Yamamura""],""type"":""article"",""altmetric_id"":73537965,""schema"":""1.5.4"",""is_oa"":false,""publisher_subjects"":[{""name"":""Intensive / Critical Care Medicine"",""scheme"":""springer""},{""name"":""Medicine & Public Health"",""scheme"":""springer""}],""cited_by_posts_count"":6,""cited_by_tweeters_count"":5,""cited_by_accounts_count"":5,""last_updated"":1624374589,""score"":2.25,""history"":{""1y"":1,""6m"":0,""3m"":0,""1m"":0,""1w"":0,""6d"":0,""5d"":0,""4d"":0,""3d"":0,""2d"":0,""1d"":0,""at"":2.25},""url"":""http://dx.doi.org/10.1186/s40560-019-0415-z"",""added_on"":1578022157,""published_on"":1577923200,""readers"":{""citeulike"":""0"",""mendeley"":""34"",""connotea"":""0""},""readers_count"":34,""images"":{""small"":""https://badges.altmetric.com/?size=64&score=3&types=tttttttt"",""medium"":""https://badges.altmetric.com/?size=100&score=3&types=tttttttt"",""large"":""https://badges.altmetric.com/?size=180&score=3&types=tttttttt""},""details_url"":""https://www.altmetric.com/details.php?citation_id=73537965""}"
1,https://api.altmetric.com/v1/doi/10.1186/s40635-019-0284-8,f73c639r,Not Found
2,https://api.altmetric.com/v1/doi/10.1186/s12864-019-6400-z,1qgpa45q,"{""title"":""Whole genome sequencing and phylogenetic analysis of human metapneumovirus strains from Kenya and Zambia"",""doi"":""10.1186/s12864-019-6400-z"",""pmid"":""31898474"",""isbns"":[],""altmetric_jid"":""4f6fa4d03cf058f6100007b9"",""issns"":[""1471-2164""],""journal"":""BMC Genomics"",""cohorts"":{""pub"":1},""context"":{""all"":{""count"":17882239,""mean"":9.484234012868734,""rank"":15805480,""pct"":1,""higher_than"":0},""journal"":{""count"":9453,""mean"":4.380810536337678,""rank"":8159,""pct"":1,""higher_than"":0},""similar_age_3m"":{""count"":396146,""mean"":12.997576620739943,""rank"":330174,""pct"":1,""higher_than"":0},""similar_age_journal_3m"":{""count"":817,""mean"":4.111747858017138,""rank"":674,""pct"":1,""higher_than"":0}},""authors"":[""Everlyn Kamau"",""John W. Oketch"",""Zaydah R. de Laurent"",""My V. T. Phan"",""Charles N. Agoti"",""D. James Nokes"",""Matthew Cotten""],""type"":""article"",""handles"":[],""altmetric_id"":73584247,""schema"":""1.5.4"",""is_oa"":false,""publisher_subjects"":[{""name"":""Animal Genetics and Genomics"",""scheme"":""springer""},{""name"":""Life Sciences"",""scheme"":""springer""},{""name"":""Life Sciences, general"",""scheme"":""springer""},{""name"":""Microarrays"",""scheme"":""springer""},{""name"":""Microbial Genetics and Genomics"",""scheme"":""springer""},{""name"":""Plant Genetics and Genomics"",""scheme"":""springer""},{""name"":""Proteomics"",""scheme"":""springer""},{""name"":""Biological Sciences"",""scheme"":""era""},{""name"":""Medical And Health Sciences"",""scheme"":""era""},{""name"":""Information And Computing Sciences"",""scheme"":""era""}],""cited_by_posts_count"":1,""cited_by_tweeters_count"":1,""cited_by_accounts_count"":1,""last_updated"":1591266873,""score"":0.25,""histor